In [1]:
# Operations are defined by manipulating registers with 3 operands.
# The following array of operations is defined by the opcode 
# and lambda functions that takes the registers as first argument.
# for readability purpose, immediate arguments is labeled as `i`
# while ignored argument is labeled as `z`.
#
# Since Julia uses 1-based index, the operands must be added by 1
# to correctly reference the register.
ops = [
    (:addr, (x, a, b, c) -> x[c+1] = x[a+1] + x[b+1]),
    (:addi, (x, a, i, c) -> x[c+1] = x[a+1] + i),
    (:mulr, (x, a, b, c) -> x[c+1] = x[a+1] * x[b+1]),
    (:muli, (x, a, i, c) -> x[c+1] = x[a+1] * i),
    (:banr, (x, a, b, c) -> x[c+1] = x[a+1] & x[b+1]),
    (:bani, (x, a, i, c) -> x[c+1] = x[a+1] & i),
    (:borr, (x, a, b, c) -> x[c+1] = x[a+1] | x[b+1]),
    (:bori, (x, a, i, c) -> x[c+1] = x[a+1] | i),
    (:setr, (x, a, z, c) -> x[c+1] = x[a+1]),
    (:seti, (x, i, z, c) -> x[c+1] = i),
    (:gtir, (x, i, b, c) -> x[c+1] = i > x[b+1] ? 1 : 0),
    (:gtri, (x, a, i, c) -> x[c+1] = x[a+1] > i ? 1 : 0),
    (:gtrr, (x, a, b, c) -> x[c+1] = x[a+1] > x[b+1] ? 1 : 0),
    (:eqir, (x, i, b, c) -> x[c+1] = i == x[b+1] ? 1 : 0),
    (:eqri, (x, a, i, c) -> x[c+1] = x[a+1] == i ? 1 : 0),
    (:eqrr, (x, a, b, c) -> x[c+1] = x[a+1] == x[b+1] ? 1 : 0)
];
opsdct = Dict(op[1] => op[2] for op in ops)

Dict{Symbol,Function} with 16 entries:
  :gtrr => ##15#31()
  :mulr => ##5#21()
  :borr => ##9#25()
  :seti => ##12#28()
  :eqri => ##17#33()
  :eqrr => ##18#34()
  :gtri => ##14#30()
  :muli => ##6#22()
  :gtir => ##13#29()
  :setr => ##11#27()
  :addr => ##3#19()
  :banr => ##7#23()
  :eqir => ##16#32()
  :addi => ##4#20()
  :bori => ##10#26()
  :bani => ##8#24()

In [2]:
mutable struct Register
    r
    rx
    ip
end

In [89]:
# return true when it's finished, false otherwise
function execute!(reg, lines)
    # exit when it's outside program
    reg.ip + 1 > length(lines) && return true
    
    # parse instruction at line `ipaddr`
    v = split(lines[reg.ip + 1], " ") 
    i, a, b, c = Symbol(v[1]), parse(Int, v[2]), parse(Int, v[3]), parse(Int, v[4])
    
    # save ip to the designated register
    reg.r[reg.rx + 1] = reg.ip

    # execute instruction
    opsdct[i](reg.r, a, b, c)
    
    # restore ip
    reg.ip =  reg.r[reg.rx + 1]
    reg.ip += 1

    false
end

function runprog(s, rx, registers)
    lines = split(s, "\n")
    reg = Register(registers, rx, 0)
    cnt = 0
    while true
        done = execute!(reg, lines)
        done && break
        cnt += 1
        cnt > 100_000_000 && break
    end
    (reg, cnt)
end

runprog (generic function with 6 methods)

In [90]:
lines = """seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5"""
runprog(lines, 0, fill(0, 6))

(Register([6, 5, 6, 0, 0, 9], 0, 7), 5)

In [91]:
# part 1
lines = """addi 3 16 3
seti 1 5 2
seti 1 5 4
mulr 2 4 1
eqrr 1 5 1
addr 1 3 3
addi 3 1 3
addr 2 0 0
addi 4 1 4
gtrr 4 5 1
addr 3 1 3
seti 2 7 3
addi 2 1 2
gtrr 2 5 1
addr 1 3 3
seti 1 4 3
mulr 3 3 3
addi 5 2 5
mulr 5 5 5
mulr 3 5 5
muli 5 11 5
addi 1 3 1
mulr 1 3 1
addi 1 12 1
addr 5 1 5
addr 3 0 3
seti 0 5 3
setr 3 4 1
mulr 1 3 1
addr 3 1 1
mulr 3 1 1
muli 1 14 1
mulr 1 3 1
addr 5 1 5
seti 0 9 0
seti 0 4 3"""
runprog(lines, 3, fill(0, 6))

(Register([1374, 1, 915, 256, 915, 914], 3, 257), 6686836)

In [92]:
# part 2 - more flexible and debug logging
function runprog(s, rx, registers, startip, stopat, debug = false)
    lines = split(s, "\n")
    reg = Register(registers, rx, startip)
    cnt = 0
    while true
        done = execute!(reg, lines)
        done && break
        cnt += 1
        debug && println(cnt, " ip=", reg.ip, " ", reg.r)
        cnt > stopat && break
    end
    (reg, cnt)
end

runprog(lines, 3, [1,0,0,0,0,0], 0, 200, true)

1 ip=17 [1, 0, 0, 16, 0, 0]
2 ip=18 [1, 0, 0, 17, 0, 2]
3 ip=19 [1, 0, 0, 18, 0, 4]
4 ip=20 [1, 0, 0, 19, 0, 76]
5 ip=21 [1, 0, 0, 20, 0, 836]
6 ip=22 [1, 3, 0, 21, 0, 836]
7 ip=23 [1, 66, 0, 22, 0, 836]
8 ip=24 [1, 78, 0, 23, 0, 836]
9 ip=25 [1, 78, 0, 24, 0, 914]
10 ip=27 [1, 78, 0, 26, 0, 914]
11 ip=28 [1, 27, 0, 27, 0, 914]
12 ip=29 [1, 756, 0, 28, 0, 914]
13 ip=30 [1, 785, 0, 29, 0, 914]
14 ip=31 [1, 23550, 0, 30, 0, 914]
15 ip=32 [1, 329700, 0, 31, 0, 914]
16 ip=33 [1, 10550400, 0, 32, 0, 914]
17 ip=34 [1, 10550400, 0, 33, 0, 10551314]
18 ip=35 [0, 10550400, 0, 34, 0, 10551314]
19 ip=1 [0, 10550400, 0, 0, 0, 10551314]
20 ip=2 [0, 10550400, 1, 1, 0, 10551314]
21 ip=3 [0, 10550400, 1, 2, 1, 10551314]
22 ip=4 [0, 1, 1, 3, 1, 10551314]
23 ip=5 [0, 0, 1, 4, 1, 10551314]
24 ip=6 [0, 0, 1, 5, 1, 10551314]
25 ip=8 [0, 0, 1, 7, 1, 10551314]
26 ip=9 [0, 0, 1, 8, 2, 10551314]
27 ip=10 [0, 0, 1, 9, 2, 10551314]
28 ip=11 [0, 0, 1, 10, 2, 10551314]
29 ip=3 [0, 0, 1, 2, 2, 10551314]
30 ip=4 [0,

(Register([0, 0, 1, 7, 23, 10551314], 3, 8), 201)

In [105]:
# ip=4 seems somewhat special
# 46 ip=4 [0, 4, 1, 3, 4, 10551314]
# 54 ip=4 [0, 5, 1, 3, 5, 10551314]
# 62 ip=4 [0, 6, 1, 3, 6, 10551314]

# we could jump start the process towards reg5
# let's say we want to mimic:
# ip=4 [0, n, 1, 3, n, 10551314]
#   where n = 10551314
runprog(lines, 3, [0, 0, 1, 2, 20, 10551314], 3, 10, true)

1 ip=4 [0, 20, 1, 3, 20, 10551314]
2 ip=5 [0, 0, 1, 4, 20, 10551314]
3 ip=6 [0, 0, 1, 5, 20, 10551314]
4 ip=8 [0, 0, 1, 7, 20, 10551314]
5 ip=9 [0, 0, 1, 8, 21, 10551314]
6 ip=10 [0, 0, 1, 9, 21, 10551314]
7 ip=11 [0, 0, 1, 10, 21, 10551314]
8 ip=3 [0, 0, 1, 2, 21, 10551314]
9 ip=4 [0, 21, 1, 3, 21, 10551314]
10 ip=5 [0, 0, 1, 4, 21, 10551314]
11 ip=6 [0, 0, 1, 5, 21, 10551314]


(Register([0, 0, 1, 5, 21, 10551314], 3, 6), 11)

In [108]:
runprog(lines, 3, [0, 0, 1, 2, 10551313, 10551314], 3, 30, true)

1 ip=4 [0, 10551313, 1, 3, 10551313, 10551314]
2 ip=5 [0, 0, 1, 4, 10551313, 10551314]
3 ip=6 [0, 0, 1, 5, 10551313, 10551314]
4 ip=8 [0, 0, 1, 7, 10551313, 10551314]
5 ip=9 [0, 0, 1, 8, 10551314, 10551314]
6 ip=10 [0, 0, 1, 9, 10551314, 10551314]
7 ip=11 [0, 0, 1, 10, 10551314, 10551314]
8 ip=3 [0, 0, 1, 2, 10551314, 10551314]
9 ip=4 [0, 10551314, 1, 3, 10551314, 10551314]
10 ip=5 [0, 1, 1, 4, 10551314, 10551314]
11 ip=7 [0, 1, 1, 6, 10551314, 10551314]
12 ip=8 [1, 1, 1, 7, 10551314, 10551314]
13 ip=9 [1, 1, 1, 8, 10551315, 10551314]
14 ip=10 [1, 1, 1, 9, 10551315, 10551314]
15 ip=12 [1, 1, 1, 11, 10551315, 10551314]
16 ip=13 [1, 1, 2, 12, 10551315, 10551314]
17 ip=14 [1, 0, 2, 13, 10551315, 10551314]
18 ip=15 [1, 0, 2, 14, 10551315, 10551314]
19 ip=2 [1, 0, 2, 1, 10551315, 10551314]
20 ip=3 [1, 0, 2, 2, 1, 10551314]
21 ip=4 [1, 2, 2, 3, 1, 10551314]
22 ip=5 [1, 0, 2, 4, 1, 10551314]
23 ip=6 [1, 0, 2, 5, 1, 10551314]
24 ip=8 [1, 0, 2, 7, 1, 10551314]
25 ip=9 [1, 0, 2, 8, 2, 10551314]


(Register([1, 0, 2, 5, 2, 10551314], 3, 6), 31)

In [109]:
#  8 ip=3 [0, 0, 1, 2, 21, 10551314]
# 20 ip=3 [1, 0, 2, 2, 1, 10551314]
# 28 ip=3 [1, 0, 2, 2, 2, 10551314]
runprog(lines, 3, [1, 0, 2, 2, 10551313, 10551314], 3, 30, true)

1 ip=4 [1, 21102626, 2, 3, 10551313, 10551314]
2 ip=5 [1, 0, 2, 4, 10551313, 10551314]
3 ip=6 [1, 0, 2, 5, 10551313, 10551314]
4 ip=8 [1, 0, 2, 7, 10551313, 10551314]
5 ip=9 [1, 0, 2, 8, 10551314, 10551314]
6 ip=10 [1, 0, 2, 9, 10551314, 10551314]
7 ip=11 [1, 0, 2, 10, 10551314, 10551314]
8 ip=3 [1, 0, 2, 2, 10551314, 10551314]
9 ip=4 [1, 21102628, 2, 3, 10551314, 10551314]
10 ip=5 [1, 0, 2, 4, 10551314, 10551314]
11 ip=6 [1, 0, 2, 5, 10551314, 10551314]
12 ip=8 [1, 0, 2, 7, 10551314, 10551314]
13 ip=9 [1, 0, 2, 8, 10551315, 10551314]
14 ip=10 [1, 1, 2, 9, 10551315, 10551314]
15 ip=12 [1, 1, 2, 11, 10551315, 10551314]
16 ip=13 [1, 1, 3, 12, 10551315, 10551314]
17 ip=14 [1, 0, 3, 13, 10551315, 10551314]
18 ip=15 [1, 0, 3, 14, 10551315, 10551314]
19 ip=2 [1, 0, 3, 1, 10551315, 10551314]
20 ip=3 [1, 0, 3, 2, 1, 10551314]
21 ip=4 [1, 3, 3, 3, 1, 10551314]
22 ip=5 [1, 0, 3, 4, 1, 10551314]
23 ip=6 [1, 0, 3, 5, 1, 10551314]
24 ip=8 [1, 0, 3, 7, 1, 10551314]
25 ip=9 [1, 0, 3, 8, 2, 10551314]


(Register([1, 0, 3, 5, 2, 10551314], 3, 6), 31)

In [117]:
# 20 ip=3 [1, 0, 3, 2, 1, 10551314]
# 28 ip=3 [1, 0, 3, 2, 2, 10551314]
runprog(lines, 3, [1, 0, 10551313, 2, 10551313, 10551314], 3, 50, true)

1 ip=4 [1, 111330206023969, 10551313, 3, 10551313, 10551314]
2 ip=5 [1, 0, 10551313, 4, 10551313, 10551314]
3 ip=6 [1, 0, 10551313, 5, 10551313, 10551314]
4 ip=8 [1, 0, 10551313, 7, 10551313, 10551314]
5 ip=9 [1, 0, 10551313, 8, 10551314, 10551314]
6 ip=10 [1, 0, 10551313, 9, 10551314, 10551314]
7 ip=11 [1, 0, 10551313, 10, 10551314, 10551314]
8 ip=3 [1, 0, 10551313, 2, 10551314, 10551314]
9 ip=4 [1, 111330216575282, 10551313, 3, 10551314, 10551314]
10 ip=5 [1, 0, 10551313, 4, 10551314, 10551314]
11 ip=6 [1, 0, 10551313, 5, 10551314, 10551314]
12 ip=8 [1, 0, 10551313, 7, 10551314, 10551314]
13 ip=9 [1, 0, 10551313, 8, 10551315, 10551314]
14 ip=10 [1, 1, 10551313, 9, 10551315, 10551314]
15 ip=12 [1, 1, 10551313, 11, 10551315, 10551314]
16 ip=13 [1, 1, 10551314, 12, 10551315, 10551314]
17 ip=14 [1, 0, 10551314, 13, 10551315, 10551314]
18 ip=15 [1, 0, 10551314, 14, 10551315, 10551314]
19 ip=2 [1, 0, 10551314, 1, 10551315, 10551314]
20 ip=3 [1, 0, 10551314, 2, 1, 10551314]
21 ip=4 [1, 1055

(Register([10551315, 0, 10551314, 10, 5, 10551314], 3, 11), 51)

In [118]:
# 28 ip=3 [10551315, 0, 10551314, 2, 2, 10551314]
# 36 ip=3 [10551315, 0, 10551314, 2, 3, 10551314]
# 44 ip=3 [10551315, 0, 10551314, 2, 4, 10551314]
runprog(lines, 3, [10551315, 0, 10551314, 2, 10551313, 10551314], 3, 50, true)

1 ip=4 [10551315, 111330216575282, 10551314, 3, 10551313, 10551314]
2 ip=5 [10551315, 0, 10551314, 4, 10551313, 10551314]
3 ip=6 [10551315, 0, 10551314, 5, 10551313, 10551314]
4 ip=8 [10551315, 0, 10551314, 7, 10551313, 10551314]
5 ip=9 [10551315, 0, 10551314, 8, 10551314, 10551314]
6 ip=10 [10551315, 0, 10551314, 9, 10551314, 10551314]
7 ip=11 [10551315, 0, 10551314, 10, 10551314, 10551314]
8 ip=3 [10551315, 0, 10551314, 2, 10551314, 10551314]
9 ip=4 [10551315, 111330227126596, 10551314, 3, 10551314, 10551314]
10 ip=5 [10551315, 0, 10551314, 4, 10551314, 10551314]
11 ip=6 [10551315, 0, 10551314, 5, 10551314, 10551314]
12 ip=8 [10551315, 0, 10551314, 7, 10551314, 10551314]
13 ip=9 [10551315, 0, 10551314, 8, 10551315, 10551314]
14 ip=10 [10551315, 1, 10551314, 9, 10551315, 10551314]
15 ip=12 [10551315, 1, 10551314, 11, 10551315, 10551314]
16 ip=13 [10551315, 1, 10551315, 12, 10551315, 10551314]
17 ip=14 [10551315, 1, 10551315, 13, 10551315, 10551314]
18 ip=16 [10551315, 1, 10551315, 15,

(Register([10551315, 1, 10551315, 256, 10551315, 10551314], 3, 257), 19)

In [ ]:
@time runprog(lines, 3, [1,0,0,0,0,0], 0, 10551314*10551314)